In [1]:
import mdptoolbox
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as ss

In [2]:
def getAdoptMatrices(rho, underpaying):
    # creating the adopt transition & reward matrices
    adopt_transitions = np.zeros(shape = (num_states, num_states))
    adopt_rewards = np.zeros(shape = (num_states, num_states))

    # each adopt matrix only can map to (1,0,irrelevant) or (0,1,irrelevant)
    new_state_1 = (1, 0, 'irrelevant')
    new_state_2 = (0, 1, 'irrelevant')
    for state_index in range(num_states):
        a, h, fork = states[state_index]
        adopt_transitions[state_index, state_mapping[new_state_1]] = alpha
        adopt_transitions[state_index, state_mapping[new_state_2]] = 1 - alpha
        adopt_rewards[state_index, state_mapping[new_state_2]] = -1 * rho * h
        adopt_rewards[state_index, state_mapping[new_state_2]] = -1 * rho * h
    
    adjustAdopt(adopt_transitions, adopt_rewards, rho, underpaying)
    # making matrices sparse
    return ss.csr_matrix(adopt_transitions), ss.csr_matrix(adopt_rewards)

In [3]:
def getOverrideMatrices(rho):
    # creating the override transition & reward matrices
    override_transitions = np.zeros(shape = (num_states, num_states))
    override_rewards = np.zeros(shape = (num_states, num_states))

    for state_index in range(num_states):
        a, h, fork = states[state_index]
        if a > h:
            new_state_1 = (a - h, 0, 'irrelevant')
            new_state_2 = (a - h - 1, 1, 'relevant')
            override_transitions[state_index, state_mapping[new_state_1]] = alpha
            override_transitions[state_index, state_mapping[new_state_2]] = 1 - alpha
            override_rewards[state_index, state_mapping[new_state_1]] = (1 - rho) * (h + 1)
            override_rewards[state_index, state_mapping[new_state_2]] = (1 - rho) * (h + 1)
        else:
            # filling in remainder of array.
            override_transitions[state_index, 0] = 1
            override_rewards[state_index, 0] = -1 * rho * 10000
    
    forceAdopt(override_transitions, override_rewards, rho)
    return ss.csr_matrix(override_transitions), ss.csr_matrix(override_rewards)

In [4]:
def getWaitMatrices(rho):
    # creating the wait transition & reward matrices
    wait_transitions = np.zeros(shape = (num_states, num_states))
    wait_rewards = np.zeros(shape = (num_states, num_states))

    for state_index in range(num_states):
        a, h, fork = states[state_index]
        # irrelevant or relevant
        if ((fork == 'irrelevant') or (fork == 'relevant')) and (a < T) and (h < T):
            new_state_1 = (a + 1, h, 'irrelevant')
            new_state_2 = (a, h + 1, 'relevant')
            wait_transitions[state_index, state_mapping[new_state_1]] = alpha
            wait_transitions[state_index, state_mapping[new_state_2]] = 1 - alpha
        # active
        elif (fork == 'active') and (a < T) and (h < T):
            if a >= h: 
                new_state_1 = (a + 1, h, 'active')
                new_state_2 = (a - h, 1, 'relevant')
                new_state_3 = (a, h + 1, 'relevant')
                wait_transitions[state_index, state_mapping[new_state_1]] = alpha
                wait_transitions[state_index, state_mapping[new_state_2]] = gamma * (1 - alpha)
                wait_transitions[state_index, state_mapping[new_state_3]] = (1 - gamma) * (1 - alpha)
                wait_rewards[state_index, state_mapping[new_state_2]] = (1 - rho) * h
            else:
                wait_transitions[state_index, 0] = 1
                wait_rewards[state_index, 0] = -1 * rho * 10000
        else:
            wait_transitions[state_index, 0] = 1
            wait_rewards[state_index, 0] = -1 * rho * 10000

    forceAdopt(wait_transitions, wait_rewards, rho)
    return ss.csr_matrix(wait_transitions), ss.csr_matrix(wait_rewards)

In [5]:
def getMatchMatrices(rho):
    # creating the match transition & rewards matrices
    match_transitions = np.zeros(shape = (num_states, num_states))
    match_rewards = np.zeros(shape = (num_states, num_states))

    for state_index in range(num_states):
        a, h, fork = states[state_index]
        if (a >= h) and (fork == 'relevant') and (a < T) and (h < T):
            new_state_1 = (a + 1, h, 'active')
            new_state_2 = (a - h, 1, 'relevant')
            new_state_3 = (a, h + 1, 'relevant')
            match_transitions[state_index, state_mapping[new_state_1]] = alpha
            match_transitions[state_index, state_mapping[new_state_2]] = gamma * (1 - alpha)
            match_transitions[state_index, state_mapping[new_state_3]] = (1 - gamma) * (1 - alpha)
            match_rewards[state_index, state_mapping[new_state_2]] = (1 - rho) * h
        else:
            match_transitions[state_index, 0] = 1
            match_rewards[state_index, 0] = -1 * rho * 10000

    forceAdopt(match_transitions, match_rewards, rho)
    return ss.csr_matrix(match_transitions), ss.csr_matrix(match_rewards)

In [6]:
def adjustAdopt(transition_matrix, reward_matrix, rho, underpaying):
    new_state_1_index = state_mapping[(1, 0, 'irrelevant')]
    new_state_2_index = state_mapping[(0, 1, 'irrelevant')]
    for state_index in range(num_states):
        a, h, fork = states[state_index]
        if ((a == T) or (h == T)) and (h != a):
            # clear out old probabilities
            transition_matrix[state_index, :] = 0
            transition_matrix[state_index, new_state_1_index] = alpha
            transition_matrix[state_index, new_state_2_index] = 1 - alpha
            if underpaying:
                reward_matrix[state_index, new_state_1_index] = -1 * rho * h
                reward_matrix[state_index, new_state_2_index] = -1 * rho * h
            else:
                # attacker ahead
                if a > h:
                    reward_matrix[state_index, new_state_1_index] = (1-rho) * overpayAttackerAhead(a, h, rho)
                    reward_matrix[state_index, new_state_2_index] = (1-rho) * overpayAttackerAhead(a, h, rho)
                else:
                    reward_matrix[state_index, new_state_1_index] = (1-rho) * overpayHonestAhead(a, h, rho)
                    reward_matrix[state_index, new_state_2_index] = (1-rho) * overpayHonestAhead(a, h, rho)

def forceAdopt(transition_matrix, reward_matrix, rho):
    for state_index in range(num_states):
        a, h, fork = states[state_index]
        if ((a == T) or (h == T)):
            # clear out old probabilities
            transition_matrix[state_index, :] = 0
            transition_matrix[state_index, 0] = 1
            reward_matrix[state_index, 0] = -1 * rho * 10000

In [7]:
# helpers
def overpayAttackerAhead(a, h, rho):
    expr1 = (1 - rho) * (alpha * (1 - alpha)) / ((1 - 2 * alpha)**2)
    expr2 = (1/2) * (((a - h) / (1 - 2 * alpha)) + a + h)
    return expr1 + expr2

def overpayHonestAhead(a, h, rho):
    expr1 = (1 - np.power(alpha/(1-alpha), h - a)) * (-1*rho*h)
    expr2 = np.power(alpha/(1-alpha), h - a) * (1 - rho)
    expr3 = (alpha * (1-alpha)) / (np.power(1-2*alpha, 2)) + (h - a) / (1-2*alpha)
    return expr1 + expr2 * expr3

def getAllMatrices(rho, underpaying):
    adopt_t, adopt_r = getAdoptMatrices(rho, underpaying)
    override_t, override_r = getOverrideMatrices(rho)
    wait_t, wait_r = getWaitMatrices(rho)
    match_t, match_r = getMatchMatrices(rho)
    return [adopt_t, override_t, wait_t, match_t], [adopt_r, override_r, wait_r, match_r]

In [8]:
T = 5

# the numbers of states is (T+1)*(T+1)*3 because each chain can be up to T length and there are 3 fork states.
# num_states = (T+1)*(T+1)*3

# generate a state to integer mapping and list of states
state_mapping = {}
states = []
count = 0
for a in range(T+1):
    for h in range(T+1):
        for fork in ['irrelevant', 'relevant', 'active']:
            if fork == 'relevant' and h == 0:
                continue
            elif fork == 'irrelevant' and a == 0:
                continue
            state_mapping[(a, h, fork)] = count
            states.append((a, h, fork))
            count += 1

state_mapping[(0, 1, 'irrelevant')] = count
states.append((0, 1, 'irrelevant'))
num_states = count + 1

In [9]:
# initializing params
epsilon = 10e-5
gamma = 0
alpha = 0.45

In [10]:
# main algo
low = 0; high = 1
while (high - low) > epsilon / 8:
    rho = (low + high) / 2
    print(low, high, rho)
    transitions, rewards = getAllMatrices(rho, underpaying=True)
    rvi = mdptoolbox.mdp.RelativeValueIteration(transitions, rewards, epsilon/8)
    rvi.run()
    if rvi.average_reward > 0:
        low = rho
    else:
        high = rho
lower_bound = rho - epsilon
rho_prime = np.max(low - epsilon/4, 0)
transitions, rewards = getAllMatrices(rho_prime, underpaying=False)
rvi = mdptoolbox.mdp.RelativeValueIteration(transitions, rewards, epsilon)
rvi.run()
upper_bound = rho_prime + 2 * (rvi.average_reward + epsilon)
print(alpha)
print("upper bound: ", upper_bound)
print("lower bound: ", lower_bound)

0 1 0.5
0.5 1 0.75
0.5 0.75 0.625
0.625 0.75 0.6875
0.625 0.6875 0.65625
0.65625 0.6875 0.671875
0.671875 0.6875 0.6796875
0.671875 0.6796875 0.67578125
0.671875 0.67578125 0.673828125
0.671875 0.673828125 0.6728515625
0.671875 0.6728515625 0.67236328125
0.67236328125 0.6728515625 0.672607421875
0.672607421875 0.6728515625 0.6727294921875
0.672607421875 0.6727294921875 0.67266845703125
0.672607421875 0.67266845703125 0.672637939453125
0.672637939453125 0.67266845703125 0.6726531982421875
0.672637939453125 0.6726531982421875 0.6726455688476562
0.45
upper bound:  1.9005237977857337
lower bound:  0.6725455688476563


In [103]:
rho = 0.7853927612304688
low = 0.7853851318359375
lower_bound = rho - epsilon
rho_prime = np.max(low - epsilon/4, 0)
transitions, rewards = getAllMatrices(rho_prime, underpaying=True)
rvi = mdptoolbox.mdp.RelativeValueIteration(transitions, rewards, epsilon)
rvi.run()
upper_bound = rho_prime + 2 * (rvi.average_reward + epsilon)
print("upper bound: ", upper_bound)
print("lower bound: ", lower_bound)

upper bound:  0.7739318652956764
lower bound:  0.7852927612304688


In [34]:
index = state_mapping[(4,2,'irrelevant')]
states[index], rvi.policy[index]

((4, 2, 'irrelevant'), 1)

In [39]:
rvi.average_reward

0.32686855675173965

In [12]:
# 0 1 0.5
# 0.5 1 0.75
# 0.75 1 0.875
# 0.75 0.875 0.8125
# 0.75 0.8125 0.78125
# 0.78125 0.8125 0.796875
# 0.78125 0.796875 0.7890625
# 0.78125 0.7890625 0.78515625
# 0.78515625 0.7890625 0.787109375
# 0.78515625 0.787109375 0.7861328125
# 0.78515625 0.7861328125 0.78564453125
# 0.78515625 0.78564453125 0.785400390625
# 0.78515625 0.785400390625 0.7852783203125
# 0.7852783203125 0.785400390625 0.78533935546875
# 0.78533935546875 0.785400390625 0.785369873046875
# 0.785369873046875 0.785400390625 0.7853851318359375
# 0.7853851318359375 0.785400390625 0.7853927612304688
# 0.45
# upper bound:  0.9358516523294126
# lower bound:  0.7852927612304688